In [ ]:
import boto3
from datetime import datetime

# Print execution info
print(f"Execution started at: {datetime.now()}")

# Get current IAM role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"Current Role ARN: {identity['Arn']}")
print(f"Account ID: {identity['Account']}")

In [ ]:
# Parameters cell - will be replaced by Papermill
model_s3_uri = None
sklearn_version = "1.2-1"
python_version = "py3"
inference_instance_type = "ml.m5.large"


In [ ]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

In [ ]:
from datetime import datetime
print(f"Execution Start Time (UTC): {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}")


In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')


In [ ]:
import boto3
import sagemaker
from sagemaker_studio import Project
from sagemaker.sklearn import SKLearnModel

print("✅ Imports successful")

In [ ]:
from sagemaker_studio import Project

# Get project connections dynamically
proj = Project()

# Get region
region = boto3.Session().region_name
print(f"✅ Region: {region}")

# Get S3 bucket
s3_shared_conn = proj.connection('default.s3_shared')
bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]
print(f"✅ S3 Bucket: {bucket}")

# Get IAM role
iam_conn = proj.connection('default.iam')
role = iam_conn.iam_role
print(f"✅ IAM Role: {role}")

# Create SageMaker session
session = sagemaker.Session()


In [ ]:
# Cleanup old endpoints from previous runs
print("\n🧹 Cleaning up old endpoints...")

try:
    sm_client = boto3.client('sagemaker', region_name=region)
    endpoints = sm_client.list_endpoints(NameContains='sagemaker-scikit-learn', MaxResults=10)
    
    for endpoint in endpoints['Endpoints']:
        endpoint_name = endpoint['EndpointName']
        try:
            print(f"   Deleting {endpoint_name}...")
            sm_client.delete_endpoint(EndpointName=endpoint_name)
            sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
        except Exception as e:
            print(f"   Could not delete {endpoint_name}: {e}")
    
    if not endpoints['Endpoints']:
        print("   No old endpoints to clean up")
    else:
        print(f"   ✅ Cleaned up {len(endpoints['Endpoints'])} endpoints")
except Exception as e:
    print(f"   ⚠️  Cleanup warning: {e}")

In [ ]:
# Get model from S3
print("\n📦 Using model from S3...")

if not model_s3_uri:
    raise ValueError("model_s3_uri parameter is required")

model_data = model_s3_uri
print(f"   Model S3 URI: {model_data}")
print("   ✅ Model location retrieved")

In [ ]:
# Deploy model
print("\n🚀 Deploying model...")

# Download inference.py from S3 to temp directory
import tempfile
import os

code_dir = tempfile.mkdtemp()
inference_file = os.path.join(code_dir, 'inference.py')

s3_client = boto3.client('s3', region_name=region)
s3_client.download_file(bucket, 'shared/ml/bundle/deployment-code/inference.py', inference_file)
print(f"   Downloaded inference.py to {code_dir}")

sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    source_dir=code_dir,
    framework_version=sklearn_version,
    py_version=python_version,
    sagemaker_session=session
)

predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type
)

endpoint_name = predictor.endpoint_name
print(f"   ✅ Model deployed to endpoint: {endpoint_name}")

In [ ]:
# Test inference
print("\n🧪 Testing inference...")

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
predictor.serializer = CSVSerializer()
predictor.deserializer = CSVDeserializer()

# Test data with 20 features (matching training)
import numpy as np
test_data = np.random.randn(2, 20).tolist()

try:
    predictions = predictor.predict(test_data)
    print(f"   Test input shape: {len(test_data)}x{len(test_data[0])}")
    print(f"   Predictions: {predictions}")
    print("   ✅ Inference successful")
except Exception as e:
    print(f"   ⚠️  Inference test failed: {e}")